In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
# import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [5]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [6]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [7]:
# X.columns

In [8]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X[col_names] = features

### Applying SMOTE to the Scaled Dataset

In [9]:
X

,auction,device,time,country,ip,url,num_bids,num_first_bids,num_last_bids,time_to_bid,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,-0.298399,-0.340208,-0.132190,-0.304887,-0.135706,-0.127977,-0.102907,-0.545777,-0.529655,-0.654541,...,-0.197123,-0.103545,-0.257464,-0.298399,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,2.164678
1,-0.414523,-0.404530,-0.134389,-0.519330,-0.139281,-0.127601,-0.104346,-0.545777,-0.529655,-0.552542,...,-0.193631,1.774257,-0.257464,-0.414523,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,0.730297
2,-0.394030,-0.404530,-0.134285,-0.519330,-0.139070,-0.127601,-0.104278,-0.545777,-0.529655,-0.521954,...,-0.197123,-0.103545,-0.257464,-0.394030,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,0.730297
3,-0.414523,-0.409890,-0.134599,-0.519330,-0.139701,-0.127977,-0.104483,-0.545777,-0.529655,3.159798,...,-0.197123,-0.103545,-0.257464,-0.414523,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,2.164678
4,-0.264244,-0.131162,-0.118474,-0.476442,-0.114046,-0.094108,-0.093930,-0.545777,-0.529655,-0.852382,...,-0.127279,-0.103545,0.009616,-0.264244,-0.196489,-0.106261,0.114687,-0.369671,-0.370524,-0.672558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,-0.250583,-0.393809,-0.131248,-0.390664,-0.138860,-0.127601,-0.102085,1.832251,-0.529655,-0.077373,...,-0.197123,-0.103545,-0.257464,-0.250583,-0.196489,-0.130120,0.114687,-0.369671,-0.370524,0.730297
2009,-0.414523,-0.409890,-0.134599,-0.519330,-0.139701,-0.127977,-0.104483,-0.545777,-0.529655,3.090534,...,-0.197123,-0.103545,-0.257464,-0.414523,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,-0.704083
2010,-0.414523,-0.404530,-0.134494,-0.519330,-0.139491,-0.127977,-0.104415,-0.545777,-0.529655,-0.506158,...,-0.197123,-0.103545,-0.257464,-0.414523,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,-0.704083
2011,-0.414523,-0.409890,-0.134599,-0.519330,-0.139701,-0.127977,-0.104483,-0.545777,-0.529655,-0.707190,...,-0.197123,-0.103545,-0.257464,-0.414523,-0.213799,-0.404496,0.114687,-0.369671,-0.370524,-0.704083


### Final Features

### Traaining all models on using train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
sm = SMOTE(sampling_strategy='minority', random_state = 42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [11]:
selected_features = ['num_bids_per_device', 'percentage_of_auctions_above_threshold',
                     'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 
                     'num_bids_per_country', 'auction', 'max_devices_per_auction', 
                     'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold',
                     'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction',
                     'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy',
                     'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time',
                     'std_url_per_auction']


X = X[selected_features]

In [12]:
len(X.columns)

25

### Applying StratifiedKFold

In [13]:
def evaluate(model, X_train, y_train, X_test, y_test):
    print("TRAIN")
    train_predictions = model.predict_proba(X_train)
    train_pred = model.predict(X_train)
        
    train_accuracy = accuracy_score(y_train, train_pred)
    train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])
    train_fbeta = fbeta_score(y_train, train_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_train, train_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(train_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
    print("*" * 100)
    
    print("TEST")
    
    test_predictions = model.predict_proba(X_test)
    test_pred = model.predict(X_test)
        
    test_accuracy = accuracy_score(y_test, test_pred)
    test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])
    test_fbeta = fbeta_score(y_test, test_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_test, test_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(test_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
    print("*" * 100)
    
    return [train_accuracy, train_auc_roc_score, train_fbeta, test_accuracy, test_auc_roc_score, test_fbeta]

In [14]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### Base Models

In [15]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
counter = 1
xgb_accuracy = []

over = SMOTE(sampling_strategy=0.1, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.5, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for train_index, test_index in skf.split(X, y):
    print("Fold number " + str(counter))
    counter += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
            
    xgb = XGBClassifier(random_state=42)

    xgb.fit(X_train, y_train)

    xgb_accuracy.append(evaluate(xgb, X_train, y_train, X_test, y_test))

Fold number 1


C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:27:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       342
         1.0     1.0000    1.0000    1.0000       171

    accuracy                         1.0000       513
   macro avg     1.0000    1.0000    1.0000       513
weighted avg     1.0000    1.0000    1.0000       513

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9831    0.9162    0.9485       191
         1.0     0.3333  

C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       342
         1.0     1.0000    1.0000    1.0000       171

    accuracy                         1.0000       513
   macro avg     1.0000    1.0000    1.0000       513
weighted avg     1.0000    1.0000    1.0000       513

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9663    0.9005    0.9322       191
         1.0     0.2083    0.4545    0.2857        11

    accuracy                         0.8762       202
   macro avg     0.5873    0.6775    0.6090       202
weighted avg     0.9250    0.8762    0.8970       202

FBeta Score
0.36764705882352944
Model Performance
Accuracy = 0.8762%.
AUC ROC = 0.8739%.
***************************

...

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       342
         1.0     1.0000    1.0000    1.0000       171

    accuracy                         1.0000       513
   macro avg     1.0000    1.0000    1.0000       513
weighted avg     1.0000    1.0000    1.0000       513

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9827    0.8901    0.9341       191
         1.0     0.2500    0.7000    0.3684        10

    accuracy                         0.8806       201
   macro avg     0.6163    0.7950    0.6512       201
weighted avg     0.9462    0.8806    0.9059       201

FBeta Score
0.5147058823529412
Model Performance
Accuracy = 0.8806%.
AUC ROC = 0.9314%.
*****************************

C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       342
         1.0     1.0000    1.0000    1.0000       171

    accuracy                         1.0000       513
   macro avg     1.0000    1.0000    1.0000       513
weighted avg     1.0000    1.0000    1.0000       513

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9826    0.8848    0.9311       191
         1.0     0.2414    0.7000    0.3590        10

    accuracy                         0.8756       201
   macro avg     0.6120    0.7924    0.6451       201
weighted avg     0.9457    0.8756    0.9027       201

FBeta Score
0.5072463768115942
Model Performance
Accuracy = 0.8756%.
AUC ROC = 0.8545%.
***********************************

C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

         0.0     1.0000    0.9971    0.9985       342
         1.0     0.9942    1.0000    0.9971       171

    accuracy                         0.9981       513
   macro avg     0.9971    0.9985    0.9978       513
weighted avg     0.9981    0.9981    0.9981       513

FBeta Score
0.9988317757009345
Model Performance
Accuracy = 0.9981%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9835    0.9372    0.9598       191
         1.0     0.3684    0.7000    0.4828        10

    accuracy                         0.9254       201
   macro avg     0.6760    0.8186    0.7213       201
weighted avg     0.9529    0.9254    0.9361       201

FBeta Score
0.5932203389830508
Model Performance
Accuracy = 0.9254%.
AUC ROC = 0.9262%.
******************************************

### Applying RandomSearch

In [16]:
def random_search_dt():
    print("RANDOM SEARCH DECISION TREE EXPERIMENT")
    criterion = ['gini', 'entropy']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [1,2,5,8]
    min_samples_leaf = [1,11,21,31]
    max_features = [5,10,15,25]
    min_impurity_decrease = [0.00005,0.0005,0.005,0.05]

    dt_random_grid = {'criterion': criterion,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'max_features': max_features,
                   'min_samples_leaf': min_samples_leaf, 
                   'min_impurity_decrease': min_impurity_decrease}
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_random = RandomizedSearchCV(estimator = dt, param_distributions = dt_random_grid, n_iter = 100, cv = 2, 
                               verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    dt_random.fit(X_train, y_train)
    
    dt_best_random = dt_random.best_estimator_
    dt_random_accuracy = evaluate(dt_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for DT")
    print(dt_random.best_params_)
    
def random_search_rf():
    print("RANDOM SEARCH RANDOM FOREST EXPERIMENT")
    # parameters for Random Forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 'sqrt',5,10,25]
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4,8,10]
    bootstrap = [True, False]

    # Create the random grid
    rf_random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    rf = RandomForestClassifier(random_state=42)
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_random_grid, n_iter = 100, cv = 2, verbose=2, 
                               random_state=42, n_jobs = -1, scoring='roc_auc')
    rf_random.fit(X_train, y_train)
    rf_best_random = rf_random.best_estimator_
    rf_random_accuracy = evaluate(rf_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for RF")
    print(rf_random.best_params_)
    

def random_search_xgb(X, y):
    print("RANDOM SEARCH XGB EXPERIMENT")
    xgb_random_grid = {'min_child_weight': range(1,10),
                    'gamma': [i/10.0 for i in range(0,10)],
                    'subsample': [i/10.0 for i in range(3,10)],
                    'colsample_bytree': [i/10.0 for i in range(6,10)],
                    'max_depth': [3, 4, 5]}
    
    skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.1, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.5, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    
    xgb = XGBClassifier(random_state=42)
    xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = xgb_random_grid, n_iter=100, scoring='roc_auc', 
                                n_jobs=-1, cv=2, random_state = 42)
    xgb_random.fit(X_train, y_train)
    xgb_best_random = xgb_random.best_estimator_
    xgb_random_accuracy = evaluate(xgb_best_random, X_train, y_train, X_test, y_test)
    print(xgb_random.best_params_)
    

In [17]:
random_search_xgb(X, y)

RANDOM SEARCH XGB EXPERIMENT
[19:27:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9925    0.9962    0.9944       266
         1.0     0.9924    0.9850    0.9887       133

    accuracy                         0.9925       399
   macro avg     0.9925    0.9906    0.9915       399
weighted avg     0.9925    0.9925    0.9925       399

FBeta Score
0.9864457831325301
Model Performance
Accuracy = 0.9925%.
AUC ROC = 0.9999%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9887    0.9133  

C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


### Applying GridSearch

# Getting results from random search to input values to grid search

## Best Random Search Param for DT
{'min_samples_split': 8, 'min_samples_leaf': 21, 'min_impurity_decrease': 0.0005, 'max_features': 25, 'max_depth': 50, 'criterion': 'entropy'}
## Best Random Search Param for RF
{'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 5, 'max_depth': 30, 'bootstrap': False}

In [20]:
def grid_search_xgb(X, y):
    print("GRID SEARCH XGB EXPERIMENT")
    xgb_grid = {
        'max_depth':[3,4,5],
        'min_child_weight':[1,3,5],
        'gamma':[0.2, 0.3, 0.4],
        'subsample':[0.8, 0.9, 1.0],
        'colsample_bytree':[0.7, 0.8, 0.9]
    }
    
    skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.1, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.5, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    xgb = XGBClassifier(random_state=42, eval_metric="error")
    xgb_grid = GridSearchCV(estimator = xgb, param_grid = xgb_grid, scoring='roc_auc',n_jobs=-1, cv=2)
    xgb_grid.fit(X_train, y_train)
    
    xgb_best_grid = xgb_grid.best_estimator_
    xgb_grid_accuracy = evaluate(xgb_best_grid, X_train, y_train, X_test, y_test)
    print(xgb_grid.best_params_)


In [21]:
grid_search_xgb(X, y)

GRID SEARCH XGB EXPERIMENT
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9925    0.9962    0.9944       266
         1.0     0.9924    0.9850    0.9887       133

    accuracy                         0.9925       399
   macro avg     0.9925    0.9906    0.9915       399
weighted avg     0.9925    0.9925    0.9925       399

FBeta Score
0.9864457831325301
Model Performance
Accuracy = 0.9925%.
AUC ROC = 0.9999%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9890    0.9307    0.9589       577
         1.0     0.3443    0.7778    0.4773        27

    accuracy                         0.9238       604
   macro avg     0.6666    0.8542    0.7181       604
weighted avg     0.9601    0.9238    0.9374       604

FBeta Score
0.621301775147929
Model Performance
Accuracy = 0.9238%.
AUC ROC

C:\Users\looya\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


# Best Params Grid Search for DT
{'criterion': 'entropy', 'max_depth': 40, 'max_features': 30, 'min_impurity_decrease': 0.0005, 'min_samples_leaf': 26, 'min_samples_split': 6}

# Best Params Grid Search for RF
{'bootstrap': False, 'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1100}

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000      1333
         1.0     1.0000    1.0000    1.0000      1333

    accuracy                         1.0000      2666
   macro avg     1.0000    1.0000    1.0000      2666
weighted avg     1.0000    1.0000    1.0000      2666

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9741    0.9792    0.9767       577
         1.0     0.5000    0.4444    0.4706        27

    accuracy                         0.9553       604
   macro avg     0.7371    0.7118    0.7236       604
weighted avg     0.9529    0.9553    0.9540       604

FBeta Score
0.45454545454545453
Model Performance
Accuracy = 0.9553%.
AUC ROC = 0.8786%.
****************************************************************************************************
Best Params Grid Search for RF
{'bootstrap': False, 'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1100}